In [14]:
from ultralytics import YOLO

# Initialize the YOLO model
custom_model = YOLO("best.pt")


In [16]:
metrics = custom_model.val(data="./dataset_yolo/drugs_dataset.yaml",split="test", plots=True)

Ultralytics YOLOv8.2.19  Python-3.10.14 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)


val: Scanning D:\UChicago\Spring Quarter 2024\Machine Learning\Final_Assignment\dataset_yolo\labels\test... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<00:00, 251.32it/s]

val: New cache created: D:\UChicago\Spring Quarter 2024\Machine Learning\Final_Assignment\dataset_yolo\labels\test.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  43%|████▎     | 3/7 [00:10<00:10,  2.56s/it]Exception in thread Thread-62 (plot_images):
Traceback (most recent call last):
  File "D:\anaconda3\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "D:\anaconda3\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "D:\anaconda3\lib\site-packages\ultralytics\utils\plotting.py", line 818, in plot_images
    mosaic = np.full((int(ns * h), int(ns * w), 3), 255, dtype=np.uint8)  # init
  File "D:\anaconda3\lib\site-packages\numpy\core\numeric.py", line 344, in full
    a = empty(shape, dtype, order)
numpy.core._exceptions._ArrayMemoryError: Unable to allocate 5.67 MiB for an array with shape (1408, 1408, 3) and data type uint8
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:11<00:00,  1.71s/it]


                   all        111        111        0.9      0.921      0.964      0.871
        Capsule_Defect        111         17       0.88      0.765      0.922      0.768
        Capsule_Normal        111         21      0.796      0.928       0.89      0.814
        Softgel_Defect        111         13      0.965          1      0.995      0.911
        Softgel_Normal        111         21       0.98          1      0.995      0.905
         Tablet_Defect        111         19      0.776          1      0.988      0.894
         Tablet_Normal        111         20          1      0.833      0.993      0.934
Speed: 0.5ms preprocess, 9.0ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs\detect\val10


In [23]:
!pip freeze

absl-py==1.4.0
accelerate @ git+https://github.com/huggingface/accelerate.git@cd7df4117d92f660965d5f737364395b5693a535
affine==2.4.0
aiohttp==3.9.5
aiosignal==1.3.1
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
altair==5.3.0
anaconda-anon-usage @ file:///C:/b/abs_c3w_h1zzjg/croot/anaconda-anon-usage_1710965204622/work
anaconda-client==1.11.2
anaconda-cloud-auth @ file:///C:/b/abs_27x7fzodlx/croot/anaconda-cloud-auth_1712794918678/work
anaconda-navigator @ file:///C:/b/abs_d8d4a02c3t/croot/anaconda-navigator_1713464063970/work
anaconda-project @ file:///C:/Windows/TEMP/abs_91fu4tfkih/croots/recipe/anaconda-project_1660339890874/work
ansi2html==1.8.0
antlr4-python3-runtime==4.9.3
anyio @ file:///C:/b/abs_847uobe7ea/croot/anyio_1706220224037/work
appdirs==1.4.4
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///C:/ci/argon2-cffi-bindings_1644569876605/work
arrow @ file:///C:/b/abs_cal7u12ktb/croot/arrow_16765881479

In [3]:
# ! pip install "ray[tune]==2.2.0"
# ! pip install "hyperopt==0.2.7"
# ! pip install "bayesian-optimization==1.3.1"

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     --------------------------------------- 20.8/20.8 MB 28.4 MB/s eta 0:00:00
     ---------------------------------------- 3.9/3.9 MB 62.0 MB/s eta 0:00:00
     ------------------------------------- 468.9/468.9 kB 28.7 MB/s eta 0:00:00
  Attempting uninstall: ray
    Found existing installation: ray 2.20.0
    Uninstalling ray-2.20.0:
      Successfully uninstalled ray-2.20.0


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'D:\\anaconda3\\Lib\\site-packages\\~ay\\thirdparty_files\\setproctitle.cp310-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 1.6/1.6 MB 7.2 MB/s eta 0:00:00
     ---------------------------------------- 200.5/200.5 kB ? eta 0:00:00


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [20]:
import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from ray.tune.search.hyperopt import HyperOptSearch
from hyperopt import hp
from ultralytics import YOLO
import numpy as np
import os



model = YOLO("D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/best.pt")
def train_model(config):
    
    # Replace this with your model training call
    results = model.train(
        data= "D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/dataset_yolo/drugs_dataset.yaml", 
        epochs=30, 
        imgsz = config["imgsz"],
        batch=config["batch_size"],
        lrf=config["lrf"]
    )
    
    # Extract the validation loss from the results
    val_loss = results["metrics"]["metrics/precision(B)"]
    tune.report(loss=val_loss)


In [21]:
ray.shutdown()
# Initialize Ray
ray.init(_temp_dir="D:/")


search_space = {
    "lrf": hp.loguniform("lrf", np.log(0.01), np.log(0.09)), # Learning rate in log scale
    "batch_size": hp.choice("batch_size", [16, 32]),  # Batch size
    "imgsz" : hp.choice("imgsz", [320, 640])
}



# Define the scheduler for early stopping
scheduler = ASHAScheduler(
    max_t=10,
    grace_period=10,
    reduction_factor=2
)

# Define the Bayesian optimization search algorithm
search_alg = HyperOptSearch(
    search_space,
    metric="metrics/precision(B)",
    mode="max"
)



analysis = tune.run(
    train_model,
    num_samples=10,  # Number of hyperparameter combinations to try
    scheduler=scheduler,
    search_alg=search_alg,
    resources_per_trial={"cpu": 4, "gpu": 1},  # Adjust based on your hardware
    metric="metrics/precision(B)",
    mode="max"
)

# Extract the best hyperparameters
best_hyperparameters = analysis.best_config
print("Best hyperparameters found were: ", best_hyperparameters)

ray.shutdown()

2024-05-23 12:30:14,752	INFO worker.py:1749 -- Started a local Ray instance.
2024-05-23 12:30:17,507	INFO tune.py:614 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(raylet) Warning: The actor ImplicitFunc is very large (11 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(train_model pid=28636) New https://pypi.org/project/ultralytics/8.2.20 available 😃 Update with 'pip install -U ultralytics'
(train_model pid=28636) Ultralytics YOLOv8.2.19 🚀 Python-3.10.14 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
(train_model pid=28636) engine\trainer: task=detect, mode=train, model=D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/best.pt, data=D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/dataset_yolo/drugs_dataset.yaml, epochs=30, time=None, patience=100, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=Fals

(train_model pid=28636) wandb: Currently logged in as: vinezhapanca (vp-projects). Use `wandb login --relogin` to force relogin
(train_model pid=28636) wandb: wandb version 0.17.0 is available!  To upgrade, please run:
(train_model pid=28636) wandb:  $ pip install wandb --upgrade
(train_model pid=28636) wandb: Tracking run with wandb version 0.16.6
(train_model pid=28636) wandb: Run data is saved locally in D:\session_2024-05-23_12-30-11_346387_21292\artifacts\2024-05-23_12-30-17\train_model_2024-05-23_12-30-17\working_dirs\train_model_ef76f731_1_batch_size=32,imgsz=640,lrf=0.0144_2024-05-23_12-30-18\wandb\run-20240523_123044-2ubkwv4u
(train_model pid=28636) wandb: Run `wandb offline` to turn off syncing.
(train_model pid=28636) wandb: Syncing run train
(train_model pid=28636) wandb:  View project at https://wandb.ai/vp-projects/YOLOv8
(train_model pid=28636) wandb:  View run at https://wandb.ai/vp-projects/YOLOv8/runs/2ubkwv4u


(train_model pid=28636) Freezing layer 'model.22.dfl.conv.weight'
(train_model pid=28636) AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
(train_model pid=28636) Downloading https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8n.pt to 'yolov8n.pt'...


  0%|          | 0.00/6.23M [00:00<?, ?B/s]
 12%|█▏        | 768k/6.23M [00:00<00:00, 7.14MB/s]
 24%|██▍       | 1.50M/6.23M [00:00<00:01, 4.82MB/s]
 34%|███▍      | 2.12M/6.23M [00:00<00:01, 3.75MB/s]
 42%|████▏     | 2.62M/6.23M [00:00<00:01, 3.43MB/s]
 48%|████▊     | 3.00M/6.23M [00:00<00:01, 3.18MB/s]
 54%|█████▍    | 3.38M/6.23M [00:01<00:00, 3.14MB/s]
 60%|██████    | 3.75M/6.23M [00:01<00:00, 3.10MB/s]
 66%|██████▌   | 4.12M/6.23M [00:01<00:00, 2.89MB/s]
 72%|███████▏  | 4.50M/6.23M [00:01<00:00, 1.83MB/s]
 76%|███████▌  | 4.75M/6.23M [00:01<00:00, 1.76MB/s]
 80%|████████  | 5.00M/6.23M [00:02<00:00, 1.67MB/s]
 84%|████████▍ | 5.25M/6.23M [00:02<00:00, 1.72MB/s]
 88%|████████▊ | 5.50M/6.23M [00:02<00:00, 1.76MB/s]
 92%|█████████▏| 5.75M/6.23M [00:02<00:00, 1.66MB/s]
 96%|█████████▋| 6.00M/6.23M [00:02<00:00, 1.57MB/s]
100%|██████████| 6.23M/6.23M [00:02<00:00, 2.26MB/s]


...

(train_model pid=26256) 
(train_model pid=26256)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/30      4.27G     0.5487     0.7115      1.087         73        640:   4%|▍         | 1/24 [00:00<00:20,  1.10it/s]
      10/30      4.28G     0.5947     0.7504      1.104         85        640:   8%|▊         | 2/24 [00:02<00:22,  1.02s/it]
      10/30      4.28G     0.5999     0.7436      1.087         85        640:  12%|█▎        | 3/24 [00:02<00:20,  1.02it/s]
      10/30      4.28G     0.5967     0.7509      1.079         88        640:  17%|█▋        | 4/24 [00:03<00:19,  1.05it/s]
      10/30      4.28G     0.5779     0.7377      1.074         79        640:  21%|██        | 5/24 [00:04<00:17,  1.07it/s]
      10/30      4.28G     0.5767     0.7379      1.079         86        640:  25%|██▌       | 6/24 [00:05<00:17,  1.03it/s]
      10/30      4.28G     0.5934     0.7631      1.098         69        640:  29%|██▉       | 7/24 [00:06<00:16,  1.05it/s]
      10/30      4.28G     0.5931     0.7607      1.098         92        640:  33%|███▎      | 8/24 [00:07<00:15,  1.

(train_model pid=26256)                    all        214        214      0.867      0.875       0.93      0.794


2024-05-23 13:35:30,068	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to 'C:/Users/vinezhapanca/ray_results/train_model_2024-05-23_12-30-17' in 0.0791s.
2024-05-23 13:35:30,106	INFO tune.py:1039 -- Total run time: 3912.60 seconds (3912.11 seconds for the tuning loop).


Best hyperparameters found were:  {'batch_size': 32, 'imgsz': 640, 'lrf': 0.059810408736502424}


In [10]:
analysis

In [22]:
ray.shutdown()
# Initialize Ray
ray.init(_temp_dir="D:/")


search_space = {
    "lrf": hp.loguniform("lrf", np.log(0.01), np.log(0.09)), # Learning rate in log scale
    "batch_size": hp.choice("batch_size", [16, 32]),  # Batch size
    "imgsz" : hp.choice("imgsz", [320, 640])
}



# Define the scheduler for early stopping
scheduler = ASHAScheduler(
    max_t=25,
    grace_period=10,
    reduction_factor=2
)

# Define the Bayesian optimization search algorithm
search_alg = HyperOptSearch(
    search_space,
    metric="metrics/precision(B)",
    mode="max"
)



analysis = tune.run(
    train_model,
    num_samples=3,  # Number of hyperparameter combinations to try
    scheduler=scheduler,
    search_alg=search_alg,
    resources_per_trial={"cpu": 4, "gpu": 1},  # Adjust based on your hardware
    metric="metrics/precision(B)",
    mode="max"
)

# Extract the best hyperparameters
best_hyperparameters = analysis.best_config
print("Best hyperparameters found were: ", best_hyperparameters)

ray.shutdown()

2024-05-23 14:09:33,329	INFO worker.py:1749 -- Started a local Ray instance.
2024-05-23 14:09:36,232	INFO tune.py:614 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(raylet) Warning: The actor ImplicitFunc is very large (11 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(train_model pid=28888) New https://pypi.org/project/ultralytics/8.2.20 available 😃 Update with 'pip install -U ultralytics'
(train_model pid=28888) Ultralytics YOLOv8.2.19 🚀 Python-3.10.14 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
(train_model pid=28888) engine\trainer: task=detect, mode=train, model=D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/best.pt, data=D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/dataset_yolo/drugs_dataset.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=Fals

(train_model pid=28888) wandb: Currently logged in as: vinezhapanca (vp-projects). Use `wandb login --relogin` to force relogin
(train_model pid=28888) wandb: wandb version 0.17.0 is available!  To upgrade, please run:
(train_model pid=28888) wandb:  $ pip install wandb --upgrade
(train_model pid=28888) wandb: Tracking run with wandb version 0.16.6
(train_model pid=28888) wandb: Run data is saved locally in D:\session_2024-05-23_14-09-29_690397_21292\artifacts\2024-05-23_14-09-37\train_model_2024-05-23_14-09-37\working_dirs\train_model_3baf0554_1_batch_size=16,imgsz=640,lrf=0.0166_2024-05-23_14-09-37\wandb\run-20240523_141006-wo63so0s
(train_model pid=28888) wandb: Run `wandb offline` to turn off syncing.
(train_model pid=28888) wandb: Syncing run train
(train_model pid=28888) wandb:  View project at https://wandb.ai/vp-projects/YOLOv8
(train_model pid=28888) wandb:  View run at https://wandb.ai/vp-projects/YOLOv8/runs/wo63so0s


(train_model pid=28888) Freezing layer 'model.22.dfl.conv.weight'
(train_model pid=28888) AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
(train_model pid=28888) Downloading https://github.com/ultralytics/assets/releases/download/v8.2.0/yolov8n.pt to 'yolov8n.pt'...


  0%|          | 0.00/6.23M [00:00<?, ?B/s]
  2%|▏         | 128k/6.23M [00:00<00:05, 1.17MB/s]
  6%|▌         | 384k/6.23M [00:00<00:04, 1.42MB/s]
 10%|█         | 640k/6.23M [00:00<00:05, 1.12MB/s]
 12%|█▏        | 768k/6.23M [00:00<00:05, 1.00MB/s]
 14%|█▍        | 896k/6.23M [00:00<00:05, 994kB/s] 
 16%|█▌        | 1.00M/6.23M [00:01<00:06, 818kB/s]
 18%|█▊        | 1.12M/6.23M [00:01<00:07, 732kB/s]
 20%|██        | 1.25M/6.23M [00:01<00:08, 630kB/s]
 22%|██▏       | 1.38M/6.23M [00:01<00:08, 566kB/s]
 24%|██▍       | 1.50M/6.23M [00:02<00:10, 479kB/s]
 26%|██▌       | 1.62M/6.23M [00:02<00:08, 550kB/s]
 28%|██▊       | 1.75M/6.23M [00:02<00:07, 654kB/s]
 30%|███       | 1.88M/6.23M [00:02<00:06, 677kB/s]
 32%|███▏      | 2.00M/6.23M [00:02<00:06, 707kB/s]
 34%|███▍      | 2.12M/6.23M [00:03<00:06, 696kB/s]
 36%|███▌      | 2.25M/6.23M [00:03<00:05, 729kB/s]
 38%|███▊      | 2.38M/6.23M [00:03<00:05, 787kB/s]
 40%|████      | 2.50M/6.23M [00:03<00:04, 881kB/s]
 44%|████▍     | 2.7

...

(train_model pid=13792) 
(train_model pid=13792)       Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/30      1.33G     0.5827      0.754     0.9955         85        320:   8%|▊         | 2/24 [00:00<00:05,  4.17it/s]
      10/30      1.33G     0.5893     0.7682     0.9889         85        320:  12%|█▎        | 3/24 [00:00<00:04,  4.74it/s]
      10/30      1.33G     0.5715     0.7708     0.9829         88        320:  17%|█▋        | 4/24 [00:00<00:05,  3.97it/s]
      10/30      1.33G     0.5566     0.7664     0.9801         79        320:  21%|██        | 5/24 [00:01<00:04,  4.13it/s]
      10/30      1.33G     0.5554     0.7734     0.9846         86        320:  25%|██▌       | 6/24 [00:01<00:05,  3.59it/s]
      10/30      1.33G     0.5712     0.7848     0.9924         70        320:  29%|██▉       | 7/24 [00:01<00:04,  3.84it/s]
      10/30      1.33G     0.5724     0.7788     0.9928         92        320:  33%|███▎      | 8/24 [00:02<00:04,  3.86it/s]
      10/30      1.33G     0.5644     0.7694     0.9877         76        320:  38%|███▊      | 9/24 [00:02<00:03,  4.

(train_model pid=13792)                    all        214        214      0.802      0.884      0.931      0.821


2024-05-23 14:32:15,636	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-05-23 14:32:15,649	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to 'C:/Users/vinezhapanca/ray_results/train_model_2024-05-23_14-09-37' in 0.0634s.
2024-05-23 14:32:15,674	INFO tune.py:1039 -- Total run time: 1359.44 seconds (1358.31 seconds for the tuning loop).


Best hyperparameters found were:  {'batch_size': 16, 'imgsz': 640, 'lrf': 0.016626029157227894}


In [1]:
from ultralytics import YOLO

#retrain with best hyperparameter {'batch_size': 16, 'imgsz': 640, 'lrf': 0.016626029157227894}

new_model = YOLO("D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/best.pt")

    
results = new_model.train(
    data= "D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/dataset_yolo/drugs_dataset.yaml", 
    epochs=30, 
    imgsz = 640,
    batch=16,
    lrf=0.016626029157227894
)

New https://pypi.org/project/ultralytics/8.2.20 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.19  Python-3.10.14 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
engine\trainer: task=detect, mode=train, model=D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/best.pt, data=D:/UChicago/Spring Quarter 2024/Machine Learning/Final_Assignment/dataset_yolo/drugs_dataset.yaml, epochs=30, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train250, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, sourc

wandb: Currently logged in as: vinezhapanca (vp-projects). Use `wandb login --relogin` to force relogin


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed 


train: Scanning D:\UChicago\Spring Quarter 2024\Machine Learning\Final_Assignment\dataset_yolo\labels\train.cache... 747 images, 0 backgrounds, 0 corrupt: 100%|██████████| 747/747 [00:00<?, ?it/s]
val: Scanning D:\UChicago\Spring Quarter 2024\Machine Learning\Final_Assignment\dataset_yolo\labels\val.cache... 214 images, 0 backgrounds, 0 corrupt: 100%|██████████| 214/214 [00:00<?, ?it/s]


Plotting labels to runs\detect\train250\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train250
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        214        214      0.958      0.975      0.985      0.891



30 epochs completed in 0.142 hours.
Optimizer stripped from runs\detect\train250\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train250\weights\best.pt, 6.2MB

Validating runs\detect\train250\weights\best.pt...
Ultralytics YOLOv8.2.19  Python-3.10.14 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.39it/s]


                   all        214        214      0.958      0.975      0.985      0.891
        Capsule_Defect        214         32      0.983      0.938      0.979      0.853
        Capsule_Normal        214         40      0.943          1      0.971      0.878
        Softgel_Defect        214         25      0.945          1      0.995      0.933
        Softgel_Normal        214         40          1      0.987      0.995      0.903
         Tablet_Defect        214         37      0.972      0.938      0.987      0.892
         Tablet_Normal        214         40      0.908      0.988      0.986      0.887
Speed: 0.7ms preprocess, 5.1ms inference, 0.0ms loss, 2.2ms postprocess per image
Results saved to runs\detect\train250


lr/pg0,▃▆███▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
lr/pg1,▃▆███▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
lr/pg2,▃▆███▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
metrics/mAP50(B),▁▁▁▄▃▂▅▄▄▄▆▆▇▇▇▇██▇▇▆██▇▇█████
metrics/mAP50-95(B),▁▃▃▄▅▄▆▅▅▄▆▆▇▇▇▇▇▇▇▇▇██▇▇█████
metrics/precision(B),▂▄▃▅▁▁▄▄▃▄▅▄▇▇▅▇▆▇▆▇▆█▆▇█▇▇███
metrics/recall(B),▄▁▃▂▄▃▅▃▇▄▆▅▄▄▇▇▇▆▄▅▄█▇▇▇█▇▇██
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▄▄▄▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▁▁▁▁▁▁▁▁▁


In [2]:
metrics_test = new_model.val(data="./dataset_yolo/drugs_dataset.yaml",split="test", plots=True)

Ultralytics YOLOv8.2.19  Python-3.10.14 torch-2.3.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Model summary (fused): 168 layers, 3006818 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning D:\UChicago\Spring Quarter 2024\Machine Learning\Final_Assignment\dataset_yolo\labels\test.cache... 111 images, 0 backgrounds, 0 corrupt: 100%|██████████| 111/111 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   0%|          | 0/7 [00:00<?, ?it/s]D:\anaconda3\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:20<00:00,  2.99s/it]


                   all        111        111      0.939      0.976      0.995      0.902
        Capsule_Defect        111         17          1      0.867      0.995      0.856
        Capsule_Normal        111         21      0.796          1      0.995      0.897
        Softgel_Defect        111         13      0.959          1      0.995      0.914
        Softgel_Normal        111         21      0.974          1      0.995        0.9
         Tablet_Defect        111         19      0.902          1      0.995      0.897
         Tablet_Normal        111         20          1      0.988      0.995      0.949
Speed: 1.1ms preprocess, 16.7ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to runs\detect\train2502
